In [1]:
import json
import os
import re
from xlsxwriter import Workbook
import easyocr
import cv2
from matplotlib import pyplot as plt
import pandas as pd
import re
import numpy as np
import Levenshtein

In [2]:
file_name = "output.xlsx"
reader = easyocr.Reader(['vi'], gpu=True)
json_folder = "./Json_result"
img_folder = "./output_images"

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [3]:
def read_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

In [4]:
def extract_text_and_position(json_data):
    extracted_data = []
    
    # Lặp qua các dòng văn bản trong 'text_lines'
    for line in json_data['data']['text_lines']:
        line_info = {
            'text': line['text'],
            'position': line['position']
        }
        
        # Thêm line_info vào kết quả
        extracted_data.append(line_info)
    
    return extracted_data

def print_text_only(extracted_data):
    data = []
    for item in extracted_data:
        data.append(str(item['text']))
    return data

In [5]:

def Is_y0(s):
    """Điều kiện để xác định loại y0."""
    return re.match(r'^(a\.|g\.|a,|g,|\d+\.$|^\d+\s\.$)$', s, re.IGNORECASE) or s in {"a", "g"}

def Is_y1(s):
    """Điều kiện để xác định loại y1."""
    return re.match(r'^(b\.)$', s, re.IGNORECASE) or s.startswith("b.") or s.startswith("b :") or s.startswith("b:") or s.startswith("b .") or s in {"b ", 
"b_", "b,", "b Phiên âm:", "b. Phiên âm:", "b.Phiên âm:", "b_Phiên âm:", "b Phiên âm", "b. Phiên âm", "b.Phiên âm", "b_Phiên âm", 
"b Phien am:", "b. Phien am:", "b.Phien am:", "b_Phien am:", "b Phien am", "b. Phien am", "b.Phien am", "b_Phien am", 
"Phiên âm:", "Phiên âm:", " Phiên âm:", "Phiên âm:", "Phiên âm", " Phiên âm", "Phiên âm", "Phiên âm", 
"Phien am:", " Phien am:", "Phien am:", "Phien am:", "Phien am", " Phien am", "Phien am", "Phien am"}

def Is_y2(s):
    """Điều kiện để xác định loại y1."""
    return re.match(r'^(c\.)$', s, re.IGNORECASE) or s.startswith("c.") or s.startswith("c ") or s.startswith("c :") or s.startswith("c:") or s.startswith("C.") or s.startswith("C ") or s.startswith("C:") or s.startswith("c:") or s in {"c ", 
"c_", "c,", "c Dịch nghĩa:", "c. Dịch nghĩa:", "c.Dịch nghĩa:", "c_Dịch nghĩa:", "c Dịch nghĩa", "c. Dịch nghĩa", "c.Dịch nghĩa", "c_Dịch nghĩa", 
"c Phien nghia:", "c. Phien nghia:", "c.Phien nghia:", "c_Phien nghia:", "c Phien nghia", "c. Phien nghia", "c.Phien nghia", "c_Phien nghia", 
"c Dich nghia:", "c. Dich nghia:", "c.Dich nghia:", "c_Dich nghia:", "c Dich nghia", "c. Dich nghia", "c.Dich nghia", "c_Dich nghia", "C Dich ngbia:"}


In [6]:
def remove_short_strings(strings, min_length=4):
    return [s for s in strings if len(s) >= min_length]
def is_header(text):
    return is_similar(text,"Phiên âm")
def contains_header_char(text):
    return any((is_header(text) or Is_y1(char) or Is_y2(char)) for char in text)
def remove_header_strings(strings):
    return [s for s in strings if not contains_header_char(s)]
def is_similar(word1, word2, threshold=5):
    if word1 == "Than ôi !" or word1 =="Than ôi!" or word1 =="Than ôi" or word1 =="Than ôi " or word1 =="Tháng" or word1 =="tháng" or word1 == "2 in" or word1 == "2 năm":
        return False
    elif word1 == "b. Phiêpâm:" or word1 == "b. Phiên âm: tông," or word1 =="b. Phlên Am:" or word1 == "b. PAMAAm:":
        return True
    distance = Levenshtein.distance(word1, word2)
    return distance <= threshold

In [7]:
def is_vietnamese(text):
    vietnamese_pattern = r'^[0-9a-zA-ZàáảãạâầấẩẫậăằắẳẵặđèéẻẽẹêềếểễệìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶĐÈÉẺẼẸÊỀẾỂỄỆÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴ]+$'
    return bool(re.fullmatch(vietnamese_pattern, text))
def contains_vietnamese_char(text):
    return any(is_vietnamese(char) for char in text)
def remove_vietnamese_strings(strings):
    return [s for s in strings if not contains_vietnamese_char(s['text'])]
def remove_punctuation(input_string):
    return re.sub(r"[:'\".,!?;-_]", "", input_string)

In [8]:
def create_DataFrame(Viet_str, Han_str):
    df = pd.DataFrame(Viet_str)
   
    return df
    

In [9]:

def process_OCR(input_folder_path):
    result = reader.readtext(input_folder_path)
    filtered_result = [
    {
        "Text": re.sub(r'_', ' ', item[1]),
        "Image box": [[int(point[0]), int(point[1])] for point in item[0]],
        "Type": "y0" if Is_y0(item[1]) else "y1" if Is_y1(item[1]) else "y2" if Is_y2(item[1]) else "unknown"
    }
    for item in result
    ]
    return filtered_result

In [10]:
def normalize_data(data):
    """
    Recursively normalizes data by converting:
    - numpy types (e.g., np.int32, np.ndarray) to standard Python types.
    - Floats to integers where applicable.
    """
    if isinstance(data, dict):
        return {key: normalize_data(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [normalize_data(item) for item in data]
    elif isinstance(data, np.ndarray):
        return [normalize_data(item) for item in data.tolist()]
    elif isinstance(data, (np.int32, np.int64, np.float32, np.float64)):
        return int(data)
    elif isinstance(data, float) and data.is_integer():
        return int(data)
    return data


def is_within_y_threshold(box1, box2, vertical_threshold=20):
    """Checks if the vertical distance between two boxes is within the given threshold."""
    y1_top = box1[0][1]
    y2_top = box2[0][1]
    return abs(y1_top - y2_top) < vertical_threshold


def extract_top_left(image_box):
    """Extracts the top-left corner coordinates (x, y) from an image box."""
    return image_box[0][0], image_box[0][1]


def sort_and_merge(data, vertical_threshold=20):
    """
    Sorts and merges text boxes based on proximity.
    
    Parameters:
    - data: The input data containing Image box, Text, and Type.
    - vertical_threshold: The maximum vertical distance to group text boxes.
    """
    normalized_data = normalize_data(data)

    df = pd.DataFrame(normalized_data, columns=['Image box', 'Text', 'Type'])
    
    if df.empty:
        return []

    coords_df = df['Image box'].apply(extract_top_left).apply(pd.Series)
    coords_df.columns = ['x1', 'y1']
    df = pd.concat([df, coords_df], axis=1)
    
    df = df.sort_values(by='y1', ascending=True)
    df['group'] = (df['y1'].diff().abs() > vertical_threshold).cumsum()
    df = df.sort_values(by=['group', 'x1'], ascending=[True, True]).reset_index(drop=True)

    merged_strings = []
    visited = [False] * len(df)

    for i in range(len(df)):
        if not visited[i]:
            current_group = [df.iloc[i]['Text']]

            if re.search(r'\d', df.iloc[i]['Text']):
                merged_strings.append(df.iloc[i]['Text'])
                visited[i] = True
                continue

            visited[i] = True

            for j in range(i + 1, len(df)):
                if not visited[j]:
                    if is_within_y_threshold(df.iloc[i]['Image box'], df.iloc[j]['Image box'], vertical_threshold):
                        if re.search(r'\d', df.iloc[j]['Text']):
                            merged_strings.append(df.iloc[j]['Text'])
                        else:
                            current_group.append(df.iloc[j]['Text'])
                        visited[j] = True

            if current_group:
                merged_text = " ".join(current_group)
                merged_strings.append(merged_text)

    return merged_strings


In [11]:
def extract_Han(input_folder_path):
    """
    Hàm xử lý các file JSON trong thư mục đầu vào để trích xuất dữ liệu Hán và Việt.

    Parameters:
    - input_folder_path: Đường dẫn tới thư mục chứa file JSON.
    - img_folder: Thư mục chứa hình ảnh OCR (dùng để xử lý đường dẫn).

    Returns:
    - Một DataFrame chứa dữ liệu đã xử lý.
    """
    data = [] 

    for file_name in os.listdir(input_folder_path):
        file_path = os.path.join(input_folder_path, file_name)
        if os.path.isfile(file_path) and file_name.endswith('.json'):
            # Đọc và xử lý dữ liệu từ JSON
            json_data = read_json_file(file_path)
            extracted_data = extract_text_and_position(json_data)
            han_text = remove_vietnamese_strings(extracted_data)

            # Xử lý thông tin trang
            page = file_name.replace("cropped_page", "").replace(".json", "").replace("_", "")
            try:
                page_num = int(page)
            except ValueError:
                print(f"Không thể chuyển đổi {page} thành số nguyên.")
                continue

            img_path = os.path.join(img_folder, f"full_page_{page_num}.png")
            img_path1 = os.path.join(img_folder, f"full_page_{page_num + 1}.png")
            print(img_path)

            viet_text = []
            if os.path.exists(img_path):
                viet_text.extend(remove_short_strings(sort_and_merge(process_OCR(img_path))))
            if os.path.exists(img_path1):
                viet_text.extend(remove_short_strings(sort_and_merge(process_OCR(img_path1))))

            viet_text1 = []
            header = False
            for s in viet_text:
                if not header:
                    if is_header(s):
                        header = True
                else:
                    viet_text1.append(s)
            viet_text1 = " ".join(viet_text1)
            viet_text1 = viet_text1.lower()
            viet_text1 = remove_punctuation(viet_text1)
            viet_text1 = viet_text1.split(" ")
            viet_text1 = [s for s in viet_text1 if s != "" and s != " "]
            viet_text1 = viet_text1[:sum(len(han['text']) for han in han_text)]  
            viet_text_idx = 0
            idx = 0
            for han in han_text:
                han_chars = list(han['text'])
                viet_words = []
                for i in range(len(han_chars)):
                    if viet_text_idx < len(viet_text1):
                        viet_words.append(viet_text1[viet_text_idx])
                        viet_text_idx += 1
                    else:
                        viet_words.append("")

                viet_text = " ".join(viet_words)
                idx += 1
                data.append({
                    "Image_name": "Sac_phong_trieu_Nguyen_tren_dia_ban_Thua_Thien_Hue_page" + str(page).zfill(3) + ".png",
                    "ID": "Sac_phong_trieu_Nguyen_tren_dia_ban_Thua_Thien_Hue." + str(page).zfill(3) + "." + str(idx).zfill(3),
                    "Image Box": han['position'],
                    "SinoNom OCR": han['text'],
                    "Chữ quốc ngữ": viet_text
                })

    return pd.DataFrame(data)


In [12]:

output = extract_Han(json_folder)

FileNotFoundError: [WinError 3] The system cannot find the path specified: './Json_result'

In [ ]:
output.to_excel('output.xlsx', index=False)

In [ ]:
output